<a href="https://colab.research.google.com/github/alexrmiller2/misc-projects/blob/main/Welcome_To_Colab_222.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import yfinance as yf

tickers = {'YMZ25.CBT': 'YMZ', '^VIX': 'VIX', 'DX-Y.NYB': 'DXY'}
data = yf.download(list(tickers.keys()), period='60d', interval='15m', group_by='ticker')
data.columns = [f"{tickers[t]}_{c}" for t, c in data.columns]

data = data.sort_index(axis=1)
data = data.drop(columns=['VIX_Volume', 'DXY_Volume']).tz_convert("US/Eastern")

data.info()

/tmp/ipython-input-1779995218.py:4: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  3 of 3 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5156 entries, 2025-08-08 03:15:00-04:00 to 2025-10-30 12:45:00-04:00
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DXY_Close   4472 non-null   float64
 1   DXY_High    4472 non-null   float64
 2   DXY_Low     4472 non-null   float64
 3   DXY_Open    4472 non-null   float64
 4   VIX_Close   3080 non-null   float64
 5   VIX_High    3080 non-null   float64
 6   VIX_Low     3080 non-null   float64
 7   VIX_Open    3080 non-null   float64
 8   YMZ_Close   3678 non-null   float64
 9   YMZ_High    3678 non-null   float64
 10  YMZ_Low     3678 non-null   float64
 11  YMZ_Open    3678 non-null   float64
 12  YMZ_Volume  3678 non-null   float64
dtypes: float64(13)
memory usage: 563.9 KB


In [117]:
import numpy as np
import pandas as pd

def kalman_filter_2d(z, dt=1.0, Q_var=1e-3, R_var=0.5):
    """
    z : list or np.array of position measurements (can contain NaNs)
    dt: time step
    Q_var: process noise variance
    R_var: measurement noise variance
    """
    # Convert to pandas Series for interpolation
    z = pd.Series(z).interpolate(method="linear").fillna(method="bfill").fillna(method="ffill").values
    n = len(z)

    xhat = np.zeros((n, 2))  # state estimates [pos, vel]

    # State transition matrix
    F = np.array([[1, dt],
                  [0, 1]])

    # Measurement matrix
    H = np.array([[1, 0]])

    # Covariances
    Q = Q_var * np.array([[dt**4/4, dt**3/2],
                          [dt**3/2, dt**2]])
    R = np.array([[R_var]])

    # Initial guesses
    xhat[0] = [z[0], 0]   # start with first measurement, zero velocity
    P = np.eye(2)         # initial covariance

    I = np.eye(2)

    for k in range(1, n):
        # --- Prediction ---
        xhat_minus = F @ xhat[k-1]
        P_minus = F @ P @ F.T + Q

        # --- Update ---
        S = H @ P_minus @ H.T + R
        K = P_minus @ H.T @ np.linalg.pinv(S)   # safer than inv
        y = z[k] - H @ xhat_minus               # innovation
        xhat[k] = xhat_minus + (K @ y).flatten()

        # Joseph form for numerical stability
        P = (I - K @ H) @ P_minus @ (I - K @ H).T + K @ R @ K.T

    return xhat

# Example usage
measurements = np.array([1, 2, 4, 7, 11, 16])  # noisy positions



def append_features(data):
    expected_columns = [
        'DXY_Close', 'DXY_High', 'DXY_Low', 'DXY_Open',
        'VIX_Close', 'VIX_High', 'VIX_Low', 'VIX_Open',
        'YMZ_Close', 'YMZ_High', 'YMZ_Low', 'YMZ_Open',
        'YMZ_Volume']

    if list(data.columns) != expected_columns:
        print("Error: Column mismatch.")
        print(f"Received columns: {list(data.columns)}")
        print(f"Expected columns: {expected_columns}")
        return data

    df = data.copy()
    df['Kalman'] = kalman_filter_2d(df['YMZ_Close'].values, dt=1, Q_var=1e-4, R_var=0.5)[:,0]
    df["SMA_20"] = df["YMZ_Close"].rolling(window=5).mean()


    return df



df = append_features(data)
import plotly.graph_objects as go
fig = go.Figure()
day = 6
data_slice = df.loc[f"2025-10-{day} 0:00":f"2025-10-{day} 23:59"].dropna()
for i, yvals in enumerate([data_slice.YMZ_Close, data_slice.Kalman, data_slice.SMA_20]):
  fig.add_trace(go.Scatter(x=data_slice.index, y=yvals))
fig.show()

/tmp/ipython-input-4035234621.py:12: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [ ]:
def compute_features(open, high, low, close):
  return